# BERTweetFullTrain.ipynb

### Designation: Model Generation Script

    Purpose: to train a single BERTweet model on the entirety of our dataset for training ('Tweets.csv')

- Requirements:
    
    Packages: tensorflow, pandas, matplotlib, transformers, sklearn

    Datasets (csv's): Tweets.csv

- This program will require an internet connection, as it will download the model and tokenizer from the HuggingFace model repository.

- Saved model-weight name (output): bertweet_FullTrained.h5
    - Please note, all files referenced (input and output) will all be on the folder-level.

## Please refer to BERTweet.ipynb for a more detailed explanation of the code.
- This code is exactly same as BERTweet, except the train-test split is disabled.
- Key parts are labeled, but please refer to BERTweet.ipynb for explanations.
- Again, GPU strongly recommended.

## 1. TensorFlow Standalone Setup

In [1]:
useCPU = False #Choose whether to use CPU or GPU for running the program

import tensorflow as tf
import os
if useCPU:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
tf.config.list_physical_devices('GPU')

Num GPUs Available:  1


2022-06-02 15:04:42.342664: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-06-02 15:04:42.353023: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-06-02 15:04:42.353588: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import InputExample, InputFeatures
with tf.device('/GPU:0'):
    model = TFAutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base",num_labels=3,problem_type="multi_label_classification")
    tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base",num_labels=3)
    model.summary()

/home/acthegreat/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-06-02 15:04:43.885255: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-02 15:04:43.886700: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-06-02 15:04:43.887114: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your 

Model: "tf_roberta_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLayer  multiple                 134309376 
 )                                                               
                                                                 
 classifier (TFRobertaClassi  multiple                 592899    
 ficationHead)                                                   
                                                                 
Total params: 134,902,275
Trainable params: 134,902,275
Non-trainable params: 0
_________________________________________________________________


## 3. Read in dataset: Tweets.csv (our dataset for training purposes), and clean up the dataset

In [4]:
dataset = pd.read_csv('../Dataset/Tweets.csv', encoding='ISO-8859-1')
dataset_drop = dataset.drop(['textID', 'selected_text'], axis=1)
dataset_drop

,text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative
...,...,...
27476,wish we could come see u on Denver husband l...,negative
27477,I`ve wondered about rake to. The client has ...,negative
27478,Yay good for both of you. Enjoy the break - y...,positive
27479,But it was worth it ****.,positive


### 3.1. Extract and encode the dataset's label column into number-category encoding.

In [5]:
datasetSentimentEncode = dataset_drop['sentiment'].apply(lambda c: 0 if c == 'negative' else (1 if c=='neutral' else 2))
datasetSentimentEncode

0        1
1        0
2        0
3        0
4        0
        ..
27476    0
27477    0
27478    2
27479    2
27480    1
Name: sentiment, Length: 27481, dtype: int64

## 4. Compiling Training/Test split dataframes

- We have disabled tran-test-split and will be passing in the entire dataset as trainDF

- commented-out-code left to show the difference between this and BERTweet.ipynb

In [7]:
#from sklearn.model_selection import train_test_split
#xtrain,xtest,ytrain,ytest = train_test_split(dataset_drop['text'].astype(str), datasetSentimentEncode, test_size=0, random_state=21)
trainDF = pd.DataFrame()
#testDF = pd.DataFrame()
trainDF['DATA_COLUMN'] = dataset_drop['text'].astype(str)
trainDF['LABEL_COLUMN'] = datasetSentimentEncode
#testDF['DATA_COLUMN'] = xtest
#testDF['LABEL_COLUMN'] = ytest
trainDF

,DATA_COLUMN,LABEL_COLUMN
0,"I`d have responded, if I were going",1
1,Sooo SAD I will miss you here in San Diego!!!,0
2,my boss is bullying me...,0
3,what interview! leave me alone,0
4,"Sons of ****, why couldn`t they put them on t...",0
...,...,...
27476,wish we could come see u on Denver husband l...,0
27477,I`ve wondered about rake to. The client has ...,0
27478,Yay good for both of you. Enjoy the break - y...,2
27479,But it was worth it ****.,2


## 5. Converting dataframes into supported input format for the AI

In [10]:
def convert_data_to_examples(train, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  '''validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)'''
  
  return train_InputExamples#, validation_InputExamples

def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )

### 5.1. The call of the functions, and batching

In [12]:
DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'
train_InputExamples = convert_data_to_examples(trainDF, DATA_COLUMN, LABEL_COLUMN)
with tf.device('/GPU:0'):
    train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
    train_data = train_data.shuffle(100).batch(32)#.repeat(2)

    '''validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
    validation_data = validation_data.batch(32)'''

/home/acthegreat/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2285: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


## 6. Train the model

In [14]:
model.layers[0].trainable = True #Testing: whether training the head only or the entire model would result in a better model.
with tf.device('/GPU:0'):
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), #default: 3e-5
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])
    model.fit(train_data, epochs=2)

Epoch 1/2
859/859 [==============================] - 472s 535ms/step - loss: 0.5696 - accuracy: 0.7630
Epoch 2/2
859/859 [==============================] - 437s 509ms/step - loss: 0.4030 - accuracy: 0.8425


## 7. (Optional) Save Weights

In [16]:
#note: this model at epoch 2 will overfit.

#model.save_weights('bertweet_FullTrained.h5')